In [1]:
#https://god.yanxishe.com/16 

import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from efficientnet_pytorch import EfficientNet

In [2]:
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.3.1
Torchvision Version:  0.4.2


In [3]:
torch.manual_seed(0)
# deterministic it can make your experiment reproducible, 
# similar to set random seed to all options where there needs a random seed.
torch.backends.cudnn.deterministic = False

#benchmark allows your network to try the best algorithm to run your implementation
torch.backends.cudnn.benchmark = True

In [4]:
train_jpg = pd.read_csv('./train.csv', names=['id', 'label'])
train_jpg['id'] = train_jpg['id'].apply(lambda x: './train/' + str(x) + '.jpg')
train_jpg 

,id,label
0,./train/0.jpg,1
1,./train/1.jpg,0
2,./train/2.jpg,0
3,./train/3.jpg,0
4,./train/4.jpg,0
...,...,...
7497,./train/7497.jpg,0
7498,./train/7498.jpg,1
7499,./train/7499.jpg,0
7500,./train/7500.jpg,1


In [5]:
class QRDataset(Dataset):
    def __init__(self, img_df,transform = None):
        self.img_df = img_df
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        start_tiem = time.time()
        
        #打开路径中的图片
        img =Image.open(self.img_df.iloc[index]['id']).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
                
                #返回的是图片，和对应的label
        return img, torch.from_numpy(np.array(self.img_df.iloc[index]['label']))
    
    def __len__(self):
        return len(self.img_df)

In [6]:
def accuracy(output, target, topk=(1,)):
    #计算准确率最高的top K个预测值。
    
     #把所有require_grad设为false，意思就是不计算梯度。
    with torch.no_grad():
        #maxk为topk中的最大值
        maxk = max(topk)
        
        #将batch_size的大小设为target一样的第一维度
        batch_size = target.size(0)
        
        #topk(求topk的值，输出维度，是否返回largest，是否要排序)
        _,pred = output.topk(maxk,1,True,True)
        
        #pred维度本为10 x maxk，将预测结果转置（拉平）成为maxk x 10
        pred = pred.t()
        
        #target的维度为10，将其变成maxk x 10，为了和pred对齐。
        #把pred和对齐后的target进行对比，输出一个maxk x 10维度的结果。
        #结果只有True和False
        correct = pred.eq(target.view(1,-1).expand_as(pred))
        
        
        res = []
        for k in topk:
            #correct_k原本维度为maxk x 10, 转为[10],并且求合，算出True的个数。
            correct_k = correct[:k].view(-1).float().sum(0, keepdim = True)
            
            #最后返回的是正确的个数，对应位相乘，correct_k和(100/batch_size)的维度必须相等
            #100/batch_size是预测的个数，res最后得到的是正确个数的百分比。
            res.append(correct_k.mul(100/batch_size))
        return res

In [7]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""


    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


In [8]:
class VisitNet(nn.Module):
    def __init__(self):
        super(VisitNet, self).__init__()
        
        # 构建神经网络
        model = models.resnet18(True)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512,2)
        self.resnet = model

#         model = EfficientNet.from_pretrained('efficientnet-b4') 
#         model._fc = nn.Linear(1792, 2)
#         self.resnet = model
        
    #前向传播
    def forward(self, img):        
        out = self.resnet(img)
        return out

In [9]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@2', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1, top5)
    
    # switch to evaluate mode
    # eval（）时，pytorch会自动把BN和DropOut固定住，不会取平均，而是用训练好的值。
    #不然的话，一旦test的batch_size过小，很容易就会被BN层导致生成图片颜色失真极大。
    model.eval()
    
    with torch.no_grad():
        end = time.time()
        
        #把data loader的数据传输到GPU上
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()
            
            #计算output
            output = model(input)
            
            #计算loss
            loss = criterion(output, target)
            
            #测量准确率，记录loss
            acc1, acc5 = accuracy(output, target, topk=(1,2))
            losses.update(loss.item(),input.size(0))
            top1.update(acc1[0], input.size(0))
             # top5.update(acc5[0], input.size(0))
                
            #计算消耗的时间
            batch_time.update(time.time() - end)
            end = time.time()
        
        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
        return top1

In [10]:
def predict(test_loader, model, tta=10):
    # switch to evaluate mode
    model.eval()
    
    # test time augmentation
    test_pred_tta = None
    
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in enumerate(test_loader):
                input = input.cuda()
                target = target.cuda()
                
                
                #计算输出
                output = model(input)
                output = output.data.cpu().numpy()
                
                
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        
        #一个输入，多多次数据变换，得到多次结果
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

In [11]:

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    # data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    # top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses, top1)
    
    #switch to train model
    #转变到训练模式
    model.train()
    
    end = time.time()
    for i, (input,target) in enumerate(train_loader):
        input = input.cuda(non_blocking = True)
        target = target.cuda(non_blocking = True)
        
        #计算输出
        output = model(input)
        loss = criterion(output, target)
        
        #计算精确率并且记录loss
        acc1, acc5 = accuracy(output, target, topk=(1,2))
        losses.update(loss.item(),input.size(0))
        top1.update(acc1[0], input.size(0))
        # top5.update(acc5[0], input.size(0))
        
        #计算梯度
        optimizer.zero_grad()
        
        #反向传播
        loss.backward()
        
        #梯度下降
        optimizer.step()
        
        # 计算消耗时间
        batch_time.update(time.time() - end)
        end = time.time()        
        
        if i % 100 == 0:
            progress.pr2int(i)       

In [12]:

def train_main():
    #使用KFold将数据分为K折
    skf = KFold(n_splits=10, random_state=233, shuffle=True)
    
    for flod_idx, (train_idx, val_idx) in enumerate(skf.split(train_jpg, train_jpg)):

        train_loader = torch.utils.data.DataLoader(
            QRDataset(train_jpg.iloc[train_idx],
                    transforms.Compose([
                                # transforms.RandomGrayscale(),
                                transforms.Resize((512, 512)),
                                transforms.RandomAffine(5),
                                # transforms.ColorJitter(hue=.05, saturation=.05),
                                # transforms.RandomCrop((88, 88)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
            ), batch_size=10, shuffle=True, num_workers=0, pin_memory=True
        )

        val_loader = torch.utils.data.DataLoader(
            QRDataset(train_jpg.iloc[val_idx],
                    transforms.Compose([
                                transforms.Resize((512, 512)),
                                # transforms.Resize((124, 124)),
                                # transforms.RandomCrop((88, 88)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
            ), batch_size=10, shuffle=False, num_workers=0, pin_memory=True
        )
        
        #设定模型
        model = VisitNet().cuda()
        # model = nn.DataParallel(model).cuda()
        
        #设定损失函数
        criterion = nn.CrossEntropyLoss().cuda()
        
        #设定优化器
        optimizer = torch.optim.SGD(model.parameters(), 0.01)
        
        #设定
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
        best_acc = 0.0

        for epoch in range(10):
            scheduler.step()
            print('Epoch: ', epoch)

            #训练
            train(train_loader, model, criterion, optimizer, epoch)
            
            #验证
            val_acc = validate(val_loader, model, criterion)
            
            #在验证集上得到的最好的准确率的参数保留为模型。
            if val_acc.avg.item() > best_acc:
                best_acc = val_acc.avg.item()
                torch.save(model.state_dict(), './resnet18_fold{0}.pt'.format(flod_idx)) 
                
if __name__ == '__main__':
    train_main()

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch:  0
[  0/676]	Time  3.563 ( 3.563)	Loss 7.3568e-01 (7.3568e-01)	Acc@1  50.00 ( 50.00)


KeyboardInterrupt: 